In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt

from sklearn.feature_selection import (VarianceThreshold, RFE, SelectFromModel)
from sklearn.svm import (SVC, LinearSVC)
from sklearn.linear_model import (LogisticRegression)
from sklearn.metrics import (roc_curve, roc_auc_score, precision_recall_curve, auc, make_scorer, 
                             recall_score, accuracy_score, precision_score, confusion_matrix)

In [9]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
cancer_df =  np.c_[cancer.data, cancer.target]
columns = np.append(cancer.feature_names, ["target"])
cancer = pd.DataFrame(cancer_df, columns=columns)

In [14]:
print(cancer.shape)
cancer[:5]

(569, 31)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


In [16]:
def leave_one_out_cv(data, filter_threshold, select_method, num_feature, svc_penalty):

    ys_test = []
    ys_predicted = []
    dec_func = []
    
    for i in data.index.values:
        
        test_data = data.loc[i]
        train_data = data.drop(index=i)
        y_test = test_data['target']
        X_test = test_data.drop(columns=['target'])
        y_train = train_data['target']
        X_train = train_data.drop(columns=['target'])
        
        # feature filtering
        X_train = X_train.loc[:, (np.count_nonzero(X_train, axis=0) > filter_threshold)]

        if select_method=='rfe':
            # select with recursive feature elimination
            selector = RFE(estimator=SVC(kernel='linear', C=1), n_features_to_select=num_feature, step=10).fit(X_train, y_train)
        
        elif select_method == 'logreg':
            # select with model: logistic regression
            selector = SelectFromModel(estimator=LogisticRegression(penalty='l1', solver='saga'), max_features=num_feature).fit(X_train, y_train)
            
        else:
            # select with model: linear svc     
            lsvc = LinearSVC(C=1, penalty='l1', dual=False)
            selector = SelectFromModel(estimator=lsvc, max_features=num_feature).fit(X_train, y_train)

        # selected features
        selected_features = X_train.columns[selector.get_support()]
        
        # transform X_train and X_test
        X_train = X_train[selected_features]
        X_test = [X_test[selected_features]]
        
        # train the model 
        classifier = SVC(C=svc_penalty, kernel='linear').fit(X_train, y_train)

        # make prediction
        y_predicted = classifier.predict(X_test)
            
        # keep lists for y_test, y_predicted, prediction accuracy (T or F), and decision functions for every LOL test
        ys_test.append(y_test)
        ys_predicted.append(y_predicted)
        dec_func.append(classifier.decision_function(X_test))
            
        # use a dictionary to keep the number of times a feature is selected   
        #for feature in selected_features:
            #if feature not in feature_count.keys():
                #feature_count[feature] = 0
            #feature_count[feature] += 1
            
    print('accuracy:', accuracy_score(ys_test, ys_predicted))
    print('precision:', precision_score(ys_test, ys_predicted))
    print('recall:', recall_score(ys_test, ys_predicted))
        
    plot_confusion_matrix(ys_test, ys_predicted, ['no cancer','cancer'])
        
    fpr, tpr, threhold = roc_curve(ys_test, dec_func, pos_label = 1)
    plot_roc_curve(fpr, tpr)
    print('Area under the roc curve:', roc_auc_score(ys_test, dec_func))
    
    return

In [ ]:
feature_score = leave_one_out_cv(cancer, filter_threshold=10, select_method='lsvc', num_feature=30, svc_penalty=10)

C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number o

C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number o

C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Jennifer Chen\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number o

In [5]:
# Helper functions

def plot_confusion_matrix(y_true, y_pred, classes, normalize=False, title=None, cmap=plt.cm.Blues):

    cm = confusion_matrix(y_true, y_pred)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def plot_roc_curve(fpr, tpr, label=None):
    plt.figure(figsize=(5,5))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation=90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")